In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
def decimate(data, dt=1):
    """
    時系列データを線形補間し、ダウンサンプリングする。

    Args:
        data (np.ndarray): 2次元配列で、最初の列が時刻、残りの列が対応するデータ値を表す。
        dt (float, optional): ダウンサンプリングのための時間間隔。デフォルトは1。

    Returns:
        np.ndarray: 指定された時間間隔で補間されたダウンサンプリングデータを含む2次元配列。各行はその時間間隔での補間値を表す。
    """

    new_data = []
    pick_time = dt

    for i in range(1, len(data)):
        if data[i, 0] > pick_time:
            x = [data[i - 1, 0], data[i, 0]]
            y = [data[i - 1, :], data[i, :]]

            a, b = np.polyfit(x, y, 1)
            interpolated_value = a * pick_time + b

            new_data.append(interpolated_value)
            pick_time += dt

    new_data = np.array(new_data)
    new_data = new_data.reshape(new_data.shape[0], -1)

    return new_data

In [ ]:
def get_gt_data(data):
    gt = decimate(pd.read_csv(data, skiprows=1).values)

    return gt

In [ ]:
def plot_time_series(data, feature_names, case_name, dataset_name):
    """
    各次元の時系列データを個別のプロットとして保存する。

    Args:
        data (np.ndarray): 2次元配列で、各列が異なる次元の時系列データを表す。
        feature_names (list): 各次元の名前を含むリスト。

    Returns:
        None
    """
    num_features = data.shape[1]

    save_dir = f"/home/dockeruser/code/notebook/svg/raw_data_{dataset_name}/case{str(case_name).zfill(4)}"
    save_dir_wo_yticks = f"/home/dockeruser/code/notebook/svg/raw_data_{dataset_name}_wo_yticks/case{str(case_name).zfill(4)}"
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(save_dir_wo_yticks, exist_ok=True)

    for i in range(1, num_features):
        # y軸のラベルを表示する
        plt.figure(figsize=(8, 4))
        sns.lineplot(x=data[:, 0], y=data[:, i])
        plt.title(f'case{str(case_name).zfill(4)}')
        plt.xlabel('Time')
        plt.ylabel(feature_names[i])
        plt.grid(False)
        plt.tight_layout()
        plt.rcParams["svg.fonttype"] = "none"
        plt.savefig(os.path.join(save_dir, f'{feature_names[i]}.svg'), format="svg")
        plt.close()

        # y軸のラベルを表示しない
        plt.figure(figsize=(8, 4))
        sns.lineplot(x=data[:, 0], y=data[:, i])
        plt.title(f'case{str(case_name).zfill(4)}')
        plt.xlabel('Time')
        plt.ylabel(feature_names[i])
        plt.yticks([])
        plt.grid(False)
        plt.tight_layout()
        plt.rcParams["svg.fonttype"] = "none"
        plt.savefig(os.path.join(save_dir_wo_yticks, f'{feature_names[i]}.svg'), format="svg")
        plt.close()

In [ ]:
for dataset_name in ["mazda-v1", "mazda-v2", "mazda-v3"]:
    gt_dir = f"/home/dockeruser/dataset/{dataset_name}"
    dataset_list = sorted(glob.glob(os.path.join(gt_dir, "*.csv")))
    feature_name = [col.split(".")[0].replace("/", "_per_") for col in pd.read_csv(dataset_list[0], skiprows=0, dtype=str).columns]
    feature_unit = [col.split(".")[0] for col in pd.read_csv(dataset_list[0], skiprows=1, dtype=str).columns]
    print(dataset_name)
    for data in tqdm(dataset_list):
        gt = get_gt_data(data)
        plot_time_series(gt, feature_name, data[-7:-4], dataset_name)

In [ ]:
nums = [[9, 41], [15, 47], [10, 36]] # num_control, num_pred
name_dict = {}
unit_dict = {}

for i, dataset_name in enumerate(["mazda-v1", "mazda-v2", "mazda-v3"]):
    gt_dir = f"/home/dockeruser/dataset/{dataset_name}"
    dataset_list = sorted(glob.glob(os.path.join(gt_dir, "*.csv")))
    feature_name = [col.split(".")[0].replace("/", "_per_") for col in pd.read_csv(dataset_list[0], skiprows=0, dtype=str).columns]
    feature_name = feature_name[1:]
    feature_unit = [col.split(".")[0] for col in pd.read_csv(dataset_list[0], skiprows=1, dtype=str).columns]
    feature_unit = feature_unit[1:]

    name_dict[dataset_name] = feature_name
    unit_dict[dataset_name] = feature_unit

    start, end = 0, 0
    print(dataset_name)
    for j in range(2):
        start = end
        end += nums[i][j]
        print(len(feature_name[start:end]), feature_name[start:end])
    print()

print(name_dict)
print(unit_dict)


In [ ]:
# 各データセットの項目名をセットに変換
set1 = set(name_dict["mazda-v1"])
set2 = set(name_dict["mazda-v2"])
set3 = set(name_dict["mazda-v3"])

# 3つのデータセットすべてに存在する項目名
all_three = set1 & set2 & set3

# 2つのデータセットにのみ存在する項目名
two_sets_1_2 = (set1 & set2) - all_three
two_sets_2_3 = (set2 & set3) - all_three
two_sets_3_1 = (set3 & set1) - all_three

# 1つのデータセットにのみ存在する項目名
one_set_1 = set1 - set2 - set3
one_set_2 = set2 - set1 - set3
one_set_3 = set3 - set1 - set2

# 結果を出力
print("3つのデータセットすべてに存在する項目名:")
print(all_three)
print("\n2つのデータセットにのみ存在する項目名:")

print("\ndata_step2 と data_refrig_only に含まれるけど、mazda1800 には含まれない項目名:")
print(two_sets_1_2)

print("\ndata_refrig_only と mazda1800 に含まれるけど、data_step2 には含まれない項目名:")
print(two_sets_2_3)

print("\nmazda1800 と data_step2 に含まれるけど、data_refrig_only には含まれない項目名:")
print(two_sets_3_1)

print("\n1つのデータセットにのみ存在する項目名:")

print("\ndata_step2 に含まれるけど、data_refrig_only と mazda1800 には含まれない項目名:")
print(one_set_1)

print("\ndata_refrig_only に含まれるけど、data_step2 と mazda1800 には含まれない項目名:")
print(one_set_2)

print("\nmazda1800 に含まれるけど、data_step2 と data_refrig_only には含まれない項目名:")
print(one_set_3)

In [ ]:
# 各データセットの項目名をセットに変換
set1 = set(name_dict["mazda-v1"])
set2 = set(name_dict["mazda-v2"])

# 2つのデータセットすべてに存在する項目名
both_sets = set1 & set2

# 1つのデータセットにのみ存在する項目名
one_set_1 = set1 - set2
one_set_2 = set2 - set1

# 結果を出力
print("2つのデータセットすべてに存在する項目名:")
print(both_sets)

print("\n1つのデータセットにのみ存在する項目名:")

print("\ndata_step2 に含まれるけど、data_refrig_only には含まれない項目名:")
print(one_set_1)

print("\ndata_refrig_only に含まれるけど、data_step2 には含まれない項目名:")
print(one_set_2)